### Vision Transformer

Load Libraries    

In [1]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from collections import Counter
import pandas as pd
import numpy as np
import tqdm
import time
import os
import sys
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import sys
from pathlib import Path

project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.data_loader import get_dataloaders
from src.log import log_experiment_results, save_loss_curve, save_confusion_matrix
from src.vit_model import build_vit_model
from src.early_stopping import EarlyStopping

# Secure W&B Login
wandb.login()

c:\Users\OMEN\anaconda3\envs\bachelor_thesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ewiehe (ewiehe-osna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

Set Parameters

In [2]:
# Experiment parameter options
pretrained_options = [True, False]
learning_rates = [0.001, 0.0001, 0.00001]
batch_sizes = [1, 8, 16, 32]
epochs = 30
augmentation = False
use_stable_diffusion = False

Load data set

In [ ]:
# Load data set based on the selected option
if use_stable_diffusion:
    data_dir = r"C:\Users\OMEN\Documents\BA_Code\stable_diffusion_data"
else:
    data_dir = r"C:\Users\OMEN\Documents\BA_Code\data"
# Load dataset (train & validation loaders)
train_loader, val_loader = get_dataloaders(data_dir, augment=augmentation)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
# Define class names
class_names = ["lie", "run", "sit", "walk_stand"]

# Define class mapping (make sure it matches your labels)
class_map = {"lie": 0, "run": 1, "sit": 2, "walk_stand": 3}

# Initialize Early Stopping
best_val_loss = float("inf")
best_conf_matrix_path = None  

In [ ]:
for pretrained in tqdm(pretrained_options, desc="Pretrained Options"):
    for lr in tqdm(learning_rates, desc="Learning Rates"):
        for batch_size in tqdm(batch_sizes, desc="Batch Sizes"):
            # Initialize W&B run
            wandb.init(
                project="animal-posture-classification",
                config={
                    "model": "ViT",
                    "pretrained": pretrained,
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "epochs": epochs,
                    "data_augmentation": augmentation,
                    "stable_diffusion": use_stable_diffusion,
                }
            )
            train_loader, val_loader = get_dataloaders(data_dir,batch_size= batch_size, augment=False)

            # Initialize model
            model = build_vit_model(pretrained=pretrained, num_classes=len(class_names)).to(device)
            
            # Define Loss Function & Optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr) 

            # Early Stopping Reset for Each Run
            early_stopping = EarlyStopping(patience=4)

            try:
                for epoch in tqdm(range(epochs), desc="Epochs"):
                    start_time = time.time()
                    model.train()
                    train_loss = 0.0

                    for images, labels in train_loader:
                        images, labels = images.to(device), labels.to(device)
                        optimizer.zero_grad()
                        outputs = model(images)
                        loss = criterion(outputs, labels)

                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) 
                        optimizer.step()

                        train_loss += loss.item()

                    train_loss /= len(train_loader)

                    # Validation
                    model.eval()
                    val_loss = 0.0
                    all_preds, all_labels = [], []

                    with torch.no_grad():
                        for images, labels in val_loader:
                            images, labels = images.to(device), labels.to(device)
                            outputs = model(images)
                            loss = criterion(outputs, labels)
                            val_loss += loss.item()

                            preds = torch.argmax(outputs, dim=1)
                            all_preds.extend(preds.cpu().numpy().tolist())
                            all_labels.extend(labels.cpu().numpy().tolist())

                    val_loss /= len(val_loader)
                    accuracy = accuracy_score(all_labels, all_preds)
                    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True, zero_division=0)

                    # Create target directory if it doesn't exist
                    conf_matrix_dir = Path("results/confusion_matrices")
                    conf_matrix_dir.mkdir(parents=True, exist_ok=True)

                    # Save and Log Confusion Matrix
                    cm = confusion_matrix(all_labels, all_preds)
                    plt.figure(figsize=(6, 6))
                    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                    plt.xlabel("Predicted")
                    plt.ylabel("True")

                    # Save to results/confusion_matrices/
                    cm_path = Path("results/confusion_matrices") / f"conf_matrix_epoch{epoch+1}.png"

                    plt.savefig(cm_path)
                    plt.close()

                    # Always log confusion matrix
                    wandb.log({
                        "epoch": epoch + 1,
                        "train_loss": train_loss,
                        "val_loss": val_loss,
                        "val_accuracy": accuracy,
                        "conf_matrix": wandb.Image(str(cm_path)),  
                        "precision": report["weighted avg"]["precision"],
                        "recall": report["weighted avg"]["recall"],
                        "f1_score": report["weighted avg"]["f1-score"],
                        "actual_epochs": epoch + 1  
                    })

                    # **Save the Best Confusion Matrix**
                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        best_conf_matrix_path = cm_path  

                    # **Check Early Stopping Condition**
                    if early_stopping(val_loss, model):  
                        print(f"Early stopping triggered at epoch {epoch + 1}. Stopping training.")
                        break


            except Exception as e:
                print(f"Error in training: {e}")
            finally:
                # **Log Best Confusion Matrix Separately (Final Run)**
                if best_conf_matrix_path:
                    wandb.log({"Confusion Matrix": wandb.Image(str(best_conf_matrix_path))}) 
                
                # Save the model
                wandb.finish()  


Pretrained Options:   0%|          | 0/2 [00:00<?, ?it/s]


Early stopping triggered. Loading best model.


Epochs:  17%|█▋        | 5/30 [07:07<35:39, 85.58s/it]

Early stopping triggered at epoch 6. Stopping training.


actual_epochs,▁▂▄▅▇█
epoch,▁▂▄▅▇█
f1_score,▁█▁▇▇▁
precision,▁█▁▇▇▁
recall,▁█▁██▁
train_loss,█▂▂▂▂▁
val_accuracy,▁█▁██▁
val_loss,█▁▃▆▄▂
actual_epochs,6
epoch,6
f1_score,0.01549


...

actual_epochs,▁▂▂▃▄▄▅▅▆▇▇█
epoch,▁▂▂▃▄▄▅▅▆▇▇█
f1_score,▁▁▁█▁▁▂▄█▃▅▅
precision,▁▁▁▆▁▁▄▅▇▅▆█
recall,▁▁▁█▁▁▂▃▇▃▅▄
train_loss,███▇█▆▅▄▃▄▁▂
val_accuracy,▁▁▁█▁▁▂▃▇▃▅▄
val_loss,▃▅▄▁▄▇▄▁▂▂▁█
actual_epochs,12
epoch,12
f1_score,0.2783


Early stopping triggered. Loading best model.


Epochs:  20%|██        | 6/30 [04:30<18:02, 45.10s/it]

Early stopping triggered at epoch 7. Stopping training.


actual_epochs,▁▂▃▅▆▇█
epoch,▁▂▃▅▆▇█
f1_score,▃▄█▁▁▁▁
precision,█▆█▁▁▁▁
recall,▃▃█▂▁▁▁
train_loss,▆▆█▃▃▅▁
val_accuracy,▃▃█▂▁▁▁
val_loss,▃▄▁█▅▅█
actual_epochs,7
epoch,7
f1_score,0.04541
